In [1]:
import pandas as pd
from os.path import join, expanduser
import sys
from datetime import datetime

In [2]:
emissions_dir = r'C:\Users\WB514197\WBG\EEAPV Pacific Observatory Files - Geospatial and Big Data\AIS\emissions'
input_dir = join(emissions_dir, '20240614')
output_dir = join(emissions_dir, 'for_charts')
df = pd.read_csv(join(input_dir, 'Emissions Pacific 201901_202405.csv'))
df.loc[:, 'ymd'] = pd.to_datetime(df[['year','month']].assign(day=1)) 
df.loc[:, 'ym'] = df['ymd'].dt.to_period('M')
df.columns = df.columns.str.lower()

In [43]:
kir_islands = ['Phoenix Group', 'Gilbert Islands', 'Line Group']
df.loc[df['country'].isin(kir_islands), 'country'] = 'Kiribati'

In [45]:
# df.to_csv(join(output_dir, 'emissions.csv'))

In [46]:
group_cols = ['country', 'ymd', 'ym'] # '_vessel_group_ais' '_w_fishing'
data_cols = ['count_vessel', 'count_vessel_day', '_missing_hours', '_total_hours']
# data_cols = ['_ch4_e', '_co_e', '_n2o_e', '_nmvoc_e', '_pm10_e', '_pm25_e', '_nox_e', '_bc_e', '_co2_f', '_sox_f', '_bc_f']

In [47]:
df_re = df.groupby(group_cols)[data_cols].sum()
df_re.reset_index(inplace=True)
# df_re.to_csv(join(output_dir, 'emissions_clean_by_vessel_group.csv'))

In [48]:
df_re.head(2)

,country,ymd,ym,count_vessel,count_vessel_day,_missing_hours,_total_hours
0,Cook Islands,2019-01-01,2019-01,1046,2229,17801.0,30445.0
1,Cook Islands,2019-02-01,2019-02,997,1825,18252.0,26144.0


In [51]:
# df_piv = df_re[group_cols+['_co2_f']].pivot(columns='_vessel_group_ais', values='_co2_f', index=['country', 'ymd', 'ym']).reset_index()
# df_piv = df_re[group_cols+['count_vessel']].pivot(columns=['_vessel_group_ais'], values='count_vessel', index=['country', 'ymd', 'ym']).reset_index()
df_piv = df_re[group_cols+['count_vessel']].pivot(columns=['country'], values='count_vessel', index=['ymd', 'ym']).reset_index()

In [68]:
order = list(df_re.groupby('country').mean().sort_values('count_vessel', ascending=False).index)

In [73]:
# df_piv.to_csv(join(output_dir, 'co2_emissions_by_vessel_group.csv'))
df_piv[['ym', 'ymd']+order].to_csv(join(output_dir, 'vessel_count_by_country.csv'), index=False)

In [56]:
# group_cols = ['country', 'ymd', 'ym']
# df_re = df.groupby(group_cols)[['_co2_f']].sum()
# df_re.reset_index(inplace=True)
# df_piv = df_re.pivot(columns='country', values='_co2_f', index='ym')